<a href="https://colab.research.google.com/github/ShibuGitB/Project-5/blob/main/Netflix_Review_Rating_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Netflix Review Rating : NLP-ANN Project***

***Importing Necessary Packages***

In [1]:
import pandas as pd
import nltk
from nltk import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import joblib

***Dataset (Kaggle)***

In [2]:
dataset=pd.read_csv("/content/drive/MyDrive/NLP Project/netflix_reviews.csv",nrows=10000)
dataset

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,c4e9d6ca-0314-495d-a2a1-90cba9ea4e89,Anne Cynthia,Nice 👍,3,0,7.120.6 build 63 35594,2024-05-27 22:45:10,7.120.6 build 63 35594
1,f1343796-c218-46be-8f36-964c27e41667,Stephanie Boo,Netflix has been crashing and crashing! you ne...,1,0,8.115.2 build 6 50688,2024-05-27 22:40:30,8.115.2 build 6 50688
2,022e3888-80d9-4228-bc3e-4d9c76768492,Allysa Lovatt,I am extremely disappointed that in order to k...,1,0,8.115.2 build 6 50688,2024-05-27 22:39:35,8.115.2 build 6 50688
3,a0d7e38f-7345-467d-a8cc-52770f5bc52f,Ahmad Bawadekji,bad,1,0,8.108.0 build 2 50642,2024-05-27 22:38:05,8.108.0 build 2 50642
4,3e0f3417-71e0-48a3-8e03-707eb05790d5,panty tera,"edit: currently cannot open the app, I get an ...",4,0,8.115.2 build 6 50688,2024-05-27 22:33:18,8.115.2 build 6 50688
...,...,...,...,...,...,...,...,...
9995,9029c0cb-a784-4f58-b051-a84a263e8b54,Jessiebon,I enjoyed Netflix for several years! Remember ...,1,0,7.66.0 build 19 35000,2023-06-20 19:19:30,7.66.0 build 19 35000
9996,b0d38b7e-9b6b-448b-aaad-b14f704969ba,Rebecca Rowland,The app is fine and it works great. I use it w...,3,108,8.30.0 build 5 40236,2022-06-14 03:10:50,8.30.0 build 5 40236
9997,ad1a7052-ac4b-416a-a89f-77c8582a2a6e,A Google user,I've only had positive experiences with Netfli...,5,34,7.30.0 build 10 34524,2019-10-17 13:16:27,7.30.0 build 10 34524
9998,dcbfb2fa-6390-43e8-bdf6-aa4c1d069ee0,A Google user,Stable enough app for an android. Should have ...,4,19,7.24.2 build 39 34469,2019-09-12 21:13:36,7.24.2 build 39 34469


***Removing unwanted Columns***

In [3]:
columns=["reviewId","userName","thumbsUpCount","reviewCreatedVersion","at","appVersion"]

for i in columns :

  dataset.drop([i],axis=1,inplace=True)

In [4]:
dataset

,content,score
0,Nice 👍,3
1,Netflix has been crashing and crashing! you ne...,1
2,I am extremely disappointed that in order to k...,1
3,bad,1
4,"edit: currently cannot open the app, I get an ...",4
...,...,...
9995,I enjoyed Netflix for several years! Remember ...,1
9996,The app is fine and it works great. I use it w...,3
9997,I've only had positive experiences with Netfli...,5
9998,Stable enough app for an android. Should have ...,4


***Checking & Handling the Missing Values***

In [5]:
dataset.isna().sum()

content    2
score      0
dtype: int64

In [6]:
mode=dataset["content"].mode()[0]
dataset["content"]=dataset["content"].fillna(mode)

In [7]:
input=dataset["content"]
input

0                                                  Nice 👍
1       Netflix has been crashing and crashing! you ne...
2       I am extremely disappointed that in order to k...
3                                                     bad
4       edit: currently cannot open the app, I get an ...
                              ...                        
9995    I enjoyed Netflix for several years! Remember ...
9996    The app is fine and it works great. I use it w...
9997    I've only had positive experiences with Netfli...
9998    Stable enough app for an android. Should have ...
9999    I'm annoyed because many of my shows that I've...
Name: content, Length: 10000, dtype: object

# ***Natural Language Processing***

***Removing Special Characters***

In [8]:
input2=input.str.replace("[^a-zA-Z0-9]","")
input2

0                                                  Nice 👍
1       Netflix has been crashing and crashing! you ne...
2       I am extremely disappointed that in order to k...
3                                                     bad
4       edit: currently cannot open the app, I get an ...
                              ...                        
9995    I enjoyed Netflix for several years! Remember ...
9996    The app is fine and it works great. I use it w...
9997    I've only had positive experiences with Netfli...
9998    Stable enough app for an android. Should have ...
9999    I'm annoyed because many of my shows that I've...
Name: content, Length: 10000, dtype: object

In [9]:
tokenizer=TweetTokenizer()

In [10]:
nltk.download('stopwords')
words=stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
stemmer=SnowballStemmer("english")

***Tokenization/Removing StopWords/Converting to Lower/Finding Root Words***

In [12]:
input3=input.apply(lambda i:[stemmer.stem(j.lower()) for j in tokenizer.tokenize(i) if j not in words]).apply(lambda j:" ".join(j))
input3

0                                                  nice 👍
1       netflix crash crash ! need fix ... lower price...
2       i extrem disappoint order keep add free plan i...
3                                                     bad
4       edit : current cannot open app , i get error c...
                              ...                        
9995    i enjoy netflix sever year ! rememb start $ 6....
9996    the app fine work great . i use i'm work kitch...
9997    i'v posit experi netflix . there suppos way as...
9998    stabl enough app android . should qualiti opti...
9999    i'm annoy mani show i'v current watch random d...
Name: content, Length: 10000, dtype: object

***Vectorization***

In [13]:
vectorizer=TfidfVectorizer()
vectorizer.fit(input3)
input=vectorizer.transform(input3)

***Converted Input By NLP***

In [14]:
print (input)

  (0, 5228)	1.0
  (1, 6000)	0.2456697590451325
  (1, 5178)	0.11216403433163737
  (1, 5143)	0.22372318682227033
  (1, 4651)	0.372782631198005
  (1, 3925)	0.5719523284003137
  (1, 3097)	0.19722958698539583
  (1, 1947)	0.6099325114333286
  (2, 8455)	0.21489506988859544
  (2, 5888)	0.23464695942953467
  (2, 5819)	0.21079188969057197
  (2, 5716)	0.30130135957702087
  (2, 5674)	0.15987510474259078
  (2, 5485)	0.2804032017718207
  (2, 5134)	0.3141692799675831
  (2, 4987)	0.18528722448277943
  (2, 4891)	0.264582004244376
  (2, 4313)	0.16379817943779412
  (2, 3365)	0.14074745636691602
  (2, 3221)	0.22503276164677272
  (2, 2918)	0.2697600585687918
  (2, 2470)	0.3189256553676048
  (2, 2317)	0.22841966493928836
  (2, 1450)	0.29713521928708747
  (2, 518)	0.20998445429566623
  :	:
  (9999, 6465)	0.1553917327404678
  (9999, 6235)	0.2562358719617542
  (9999, 6181)	0.17409901241060932
  (9999, 5841)	0.25209582625785953
  (9999, 5768)	0.09371923984947318
  (9999, 5178)	0.05786673710119621
  (9999, 5045)

In [15]:
input=input.toarray()

***Converting string output into numeric values by Label Encoder***

In [16]:
encoder=LabelEncoder()
encoder.fit(dataset["score"])
dataset["score"]=encoder.transform(dataset["score"])

In [17]:
dataset

,content,score
0,Nice 👍,2
1,Netflix has been crashing and crashing! you ne...,0
2,I am extremely disappointed that in order to k...,0
3,bad,0
4,"edit: currently cannot open the app, I get an ...",3
...,...,...
9995,I enjoyed Netflix for several years! Remember ...,0
9996,The app is fine and it works great. I use it w...,2
9997,I've only had positive experiences with Netfli...,4
9998,Stable enough app for an android. Should have ...,3


In [18]:
output=dataset["score"]
output

0       2
1       0
2       0
3       0
4       3
       ..
9995    0
9996    2
9997    4
9998    3
9999    0
Name: score, Length: 10000, dtype: int64

# ***Artificial Neural Network (ANN)***

***Applying to_categorical to Output***

In [19]:
output=to_categorical(output)

In [20]:
output=np.array(output)

***Taking input & Output for Training and Testing***

In [21]:
x_train,x_test,y_train,y_test=train_test_split(input,output,test_size=0.3,random_state=1)

***Using sequential Algorithm Creating Neuron Model Compiling & Training that***

In [22]:
obj=Sequential()

In [23]:
obj.add(Dense(5,activation="relu"))
obj.add(Dense(9,activation="relu"))
obj.add(Dense(7,activation="relu"))
obj.add(Dense(5,activation="softmax"))

In [24]:
obj.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [25]:
obj.fit(x_train,y_train,epochs=5,batch_size=20)

Epoch 1/5
350/350 [==============================] - 3s 3ms/step - loss: 1.5347 - accuracy: 0.4911
Epoch 2/5
350/350 [==============================] - 1s 3ms/step - loss: 1.4347 - accuracy: 0.4956
Epoch 3/5
350/350 [==============================] - 1s 3ms/step - loss: 1.3667 - accuracy: 0.5137
Epoch 4/5
350/350 [==============================] - 1s 3ms/step - loss: 1.1617 - accuracy: 0.6350
Epoch 5/5
350/350 [==============================] - 2s 5ms/step - loss: 1.0170 - accuracy: 0.6660


***Got the Model with Accuracy at almost 62% with epoch=5 and 1 Input layer with 5 neurons,2 Hidden layers with 9,7 neurons and a Output layers with 5 neurons***

In [26]:
obj.evaluate(x_test,y_test)

94/94 [==============================] - 1s 3ms/step - loss: 1.0784 - accuracy: 0.6290


[1.0784307718276978, 0.6290000081062317]

***Sample Prediction***

In [27]:
prediction=obj.predict(vectorizer.transform(["good"]))

1/1 [==============================] - 0s 205ms/step


In [28]:
position=prediction.argmax(axis=1)

In [29]:
rating=encoder.inverse_transform(position)
print (rating)

[5]


***Taking ANN Model,Scaler & Encoder for the Streamlit Deploy***

In [30]:
obj.save("NetFlix Review Rating Model-2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
joblib.dump(vectorizer,"NetFlix Vectorizer")

['NetFlix Vectorizer']

In [32]:
joblib.dump(encoder,"NetFlix Encoder")

['NetFlix Encoder']